In [1]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

In [6]:
b = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs[:], chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1    

In [ ]:
sorted(b.items(), key = lambda kv: kv[1], reverse=True)

In [8]:
import torch

In [16]:
N = torch.zeros(size=(28, 28), dtype=torch.int32)

In [17]:
# get the alphabet as the list of stings
chs = sorted(list(set("".join(words))))

# create character: id and id: character mappings
stoi = {s: i for i, s in enumerate(chs)}
stoi['<S>'], stoi['<E>'] = len(stoi), len(stoi) + 1
itos = {i: s for s, i in stoi.items()}

for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs[:], chs[1:]):
        ix = stoi[ch1]
        iy = stoi[ch2]
        N[ix, iy] += 1  

In [ ]:
print(N[:7, :7])